In [1]:
from google.colab import drive
drive.mount('/content/drive')
drive.mount('/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Mounted at /gdrive


In [0]:
!ln -s "/gdrive/My Drive/project/data/custom/noaa" "/content/cs"

In [3]:
cd /content/cs

/gdrive/My Drive/project/data/custom/noaa


In [5]:
!sudo pip3 install -r requirements.txt

  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=d7e8e302b158a14651aedcb958ec6aacde5812b54fb9f201b0c6cd8c71f65700
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [0]:
pwd

'/gdrive/My Drive/project/data/custom/noaa'

In [0]:
f = open("images.txt", "r")
l = f.readlines()
f.close()

ll = []
for x in l:
  xx = x.rstrip()
  xx = xx[:-4]
  ll.append(xx)

In [0]:
def train_test_split(z, test_size=0.05, random_state=0):
    import random
    random.seed(random_state)
    n = len(z)
    indexes = random.sample(range(0, n), n)
    train_size = 1 - test_size
    train_data_size = int(train_size * n)
    
    train, test = z[indexes[:train_data_size]], z[indexes[train_data_size:]]
    return train, test

import numpy as np
str_numpy = np.array(ll)



In [0]:
train, valid = train_test_split(str_numpy,random_state = 16918)

In [0]:
f2 = open("train.txt","w+")
f3 = open("valid.txt","w+")

str_train = ""
str_valid = ""

for x in train:
    str_train = str_train + x + ".jpg" + "\n"
    str_train = str_train + x + "_HF" + ".jpg" + "\n"
str_train = str_train.rstrip()

for x in valid:
    str_valid = str_valid + x + ".jpg" + "\n"
    str_valid = str_valid + x + "_HF" + ".jpg" + "\n"
str_valid = str_valid.rstrip()

f2.write(str_train)
f2.close()

f3.write(str_valid)
f3.close()


In [11]:
# anchor selection

import os
ll = []
for filename in os.listdir("/content/cs/labels"): 
    #print(filename)
    fp = "/content/cs/labels/" + filename
    f = open(fp, "r")
    l = f.readlines()
    for x in l:
      xx = x.rstrip()
      xw = xx.split()
      w = (float(xw[3]), float(xw[4]))
      ll.append(w)
len(ll)

2010

In [12]:
import random
import argparse
import numpy as np

import json

def IOU(ann, centroids):
    w, h = ann
    similarities = []

    for centroid in centroids:
        c_w, c_h = centroid

        if c_w >= w and c_h >= h:
            similarity = w*h/(c_w*c_h)
        elif c_w >= w and c_h <= h:
            similarity = w*c_h/(w*h + (c_w-w)*c_h)
        elif c_w <= w and c_h >= h:
            similarity = c_w*h/(w*h + c_w*(c_h-h))
        else: #means both w,h are bigger than c_w and c_h respectively
            similarity = (c_w*c_h)/(w*h)
        similarities.append(similarity) # will become (k,) shape

    return np.array(similarities)

def avg_IOU(anns, centroids):
    n,d = anns.shape
    sum = 0.

    for i in range(anns.shape[0]):
        sum+= max(IOU(anns[i], centroids))

    return sum/n

def print_anchors(centroids):
    out_string = ''

    anchors = centroids.copy()

    widths = anchors[:, 0]
    sorted_indices = np.argsort(widths)

    r = "anchors: ["
    for i in sorted_indices:
        out_string += str(int(anchors[i,0]*416)) + ',' + str(int(anchors[i,1]*416)) + ', '
            
    print(out_string[:-2])

def run_kmeans(ann_dims, anchor_num):
    ann_num = ann_dims.shape[0]
    iterations = 0
    prev_assignments = np.ones(ann_num)*(-1)
    iteration = 0
    old_distances = np.zeros((ann_num, anchor_num))

    indices = [random.randrange(ann_dims.shape[0]) for i in range(anchor_num)]
    centroids = ann_dims[indices]
    anchor_dim = ann_dims.shape[1]

    while True:
        distances = []
        iteration += 1
        for i in range(ann_num):
            d = 1 - IOU(ann_dims[i], centroids)
            distances.append(d)
        distances = np.array(distances) # distances.shape = (ann_num, anchor_num)

        print("iteration {}: dists = {}".format(iteration, np.sum(np.abs(old_distances-distances))))

        #assign samples to centroids
        assignments = np.argmin(distances,axis=1)

        if (assignments == prev_assignments).all() :
            return centroids

        #calculate new centroids
        centroid_sums=np.zeros((anchor_num, anchor_dim), np.float)
        for i in range(ann_num):
            centroid_sums[assignments[i]]+=ann_dims[i]
        for j in range(anchor_num):
            centroids[j] = centroid_sums[j]/(np.sum(assignments==j) + 1e-6)

        prev_assignments = assignments.copy()
        old_distances = distances.copy()

def main(annotation_dims,num_anchors):
    annotation_dims = np.array(annotation_dims)
    centroids = run_kmeans(annotation_dims, num_anchors)

    # write anchors to file
    print('\naverage IOU for', num_anchors, 'anchors:', '%0.2f' % avg_IOU(annotation_dims, centroids))
    print_anchors(centroids)


all_data = ll
main(all_data,9)


iteration 1: dists = 12330.496838800675
iteration 2: dists = 951.5758682301166
iteration 3: dists = 595.0308647401464
iteration 4: dists = 463.29111421133376
iteration 5: dists = 372.6744949151627
iteration 6: dists = 376.4905519997756
iteration 7: dists = 272.0475916601376
iteration 8: dists = 252.53406152701191
iteration 9: dists = 243.6344323393041
iteration 10: dists = 246.62111421550424
iteration 11: dists = 227.2998934242649
iteration 12: dists = 224.10340219266692
iteration 13: dists = 199.571398236183
iteration 14: dists = 225.02365538785426
iteration 15: dists = 225.68023004426374
iteration 16: dists = 181.7244412959314
iteration 17: dists = 199.51173414271818
iteration 18: dists = 143.68964054983988
iteration 19: dists = 119.39017412851436
iteration 20: dists = 130.91200588734398
iteration 21: dists = 127.94439087194962
iteration 22: dists = 138.64273224867537
iteration 23: dists = 124.2072839172605
iteration 24: dists = 120.6516641748091
iteration 25: dists = 120.44733060930